References:
 
primarily https://realpython.com/python-with-statement/, \
https://www.analyticsvidhya.com/blog/2023/01/a-beginners-guide-to-context-manager/

---
# Context Managers
---

A **context manager** allows you to allocate and release resources.

A common example of this is in file handling. When you're dealing with files, you want to ensure they are properly opened and closed, preventing any leaks or locks on the file. Here's how you might do this without a context manager:

In [4]:
file = open("hello.txt", "w")
file.write("Hello World!")
file.close()


In the example above, you have to remember to close the file by calling file.close(). If an exception occurs when you're working with the file, your program might stop running before it has the chance to close it. This can lead to data loss or other issues.

Using context managers:

In [12]:
file = open('example.txt', 'w')
try:
    file.write('Hello World!')
finally:
    file.close()

The finally block is a key feature of Python's error 
handling. No matter what happens in the try block (whether 
an exception is raised or not), the finally block is always 
executed. In this code, the finally block contains a single 
command: file.close(). This closes the file and releases any 
system resources used by it.

or

In [3]:
with open('example.txt', 'w') as file:
    file.write('Hello World!')

In this example, the with statement is used to open the file, and then it's automatically closed when the block of code is exited, even if there are any exceptions. You don't need to explicitly call file.close(), as the with statement's context manager handles that for you. This also makes your code cleaner and more readable.

Context managers are not limited to file handling. They can be used for managing any resource that needs to be explicitly cleaned up or closed, such as network connections, database sessions, or locks in threading. They provide a clean and readable way to manage resources in Python.

## The with Statement

To write a with statement, you need to use the following general syntax:

```python

with expression as variable:
    # code block

```

We saw a common use of the with statement is when working with files.

The with statement has been shown to have several valuable use cases. More and more objects in the Python standard library now provide support for the context management protocol so you can use them in a with statement.

In this section, you’ll see some examples.

**Performing High-Precision Calculations**

In [16]:
# Performing High-Precision Calculations
from decimal import Decimal, localcontext

with localcontext() as ctx:
     ctx.prec = 42
     print(Decimal("1") / Decimal("42"))

0.0238095238095238095238095238095238095238095


Unlike built-in floating-point numbers, the decimal module provides a way to adjust the precision to use in a given calculation that involves Decimal numbers. The precision defaults to 28 places, but you can change it to meet your problem requirements. A quick way to perform calculations with a custom precision is using localcontext() from decimal.

Normal output without with statement:

In [18]:
print(Decimal("1") / Decimal("42"))

0.02380952380952380952380952381


**Traversing Directories**

The os module provides a function called scandir(), which returns an iterator over os.DirEntry objects corresponding to the entries in a given directory. This function is specially designed to provide optimal performance when you’re traversing a directory structure.

A call to scandir() with the path to a given directory as an argument returns an iterator that supports the context management protocol:

In [ ]:
import os

with os.scandir(".") as entries:
    for entry in entries:
        print(entry.name, "->", entry.stat().st_size, "bytes")


In this example, you write a with statement with os.scandir() as the context manager supplier. Then you iterate over the entries in the selected directory (".") and print their name and size on the screen. In this case, .__exit__() calls scandir.close() to close the iterator and release the acquired resources. Note that if you run this on your machine, you’ll get a different output depending on the content of your current directory.

## Creating Custom Context Managers 

Creating custom context managers can be very useful for managing resources that don't have built-in context managers. There are two ways of creating them :

1. **Class Based Context Managers**:

To implement the context management protocol and create class-based context managers, you need to add both the `.__enter__()` and the `__exit__()` special methods to your classes. The table below summarizes how these methods work, the arguments they take, and the logic you can put in them:

| Method | Description |
| --- | --- |
| `.__enter__(self)` | This method handles the setup logic and is called when entering a new `with` context. Its return value is bound to the `with` target variable. |
| `.__exit__(self, exc_type, exc_value, exc_tb)` | This method handles the teardown logic and is called when the flow of execution leaves the `with` context. If an exception occurs, then `exc_type`, `exc_value`, and `exc_tb` hold the exception type, value, and traceback information, respectively. |

When the `with` statement executes, it calls `.__enter__()` on the context manager object to signal that you’re entering into a new runtime context. If you provide a target variable with the `as` specifier, then the return value of `.__enter__()` is assigned to that variable.



Here’s a sample class-based context manager that implements both methods, `.__enter__()` and `.__exit__()`. It also shows how Python calls them in a with construct:



In [26]:
class HelloContextManager:
     def __enter__(self):
         print("Entering the context...")
         return "Hello, World!"
     def __exit__(self, exc_type, exc_value, exc_tb):
         print("Leaving the context...")
         print(f"An exception occurred: {exc_type}, {exc_value}, {exc_tb}")

In [27]:
with HelloContextManager() as hello:
     print(hello)

Entering the context...
Hello, World!
Leaving the context...
An exception occurred: None, None, None


HelloContextManager implements both `.__enter__()` and `.__exit__`. In `.__enter__()`, you first print a message to signal that the flow of execution is entering a new context. Then you return the "Hello, World!" string. In `.__exit__()`, you print a message to signal that the flow of execution is leaving the context. You also print the content of its three arguments. The exc_type, exc_value, and exc_tb parameters in the `__exit__ ` method of a context manager in Python are used to handle exceptions that occur within the with statement.

**Measuring Execution Time, Class Based Context Manager**

The following example shows how to create a context manager to measure the execution time of a given code block or function:

In [47]:
from time import perf_counter

class Timer:
    def __enter__(self):
        self.start = perf_counter()
        self.end = 0.0
        return lambda: self.end - self.start

    def __exit__(self, *args):
        self.end = perf_counter()

When you use Timer in a with statement, `.__enter__()` gets called. This method uses time.perf_counter() to get the time at the beginning of the with code block and stores it in .start. It also initializes .end and returns a lambda function that computes a time delta. In this case, .start holds the initial state or time measurement.

Once the with block ends, `.__exit__()` gets called. The method gets the time at the end of the block and updates the value of .end so that the lambda function can compute the time required to run the with code block.

Here’s how you can use this context manager in your code:

In [48]:
from time import sleep

with Timer() as elapsed:
    # Time-consuming code goes here...
    sleep(0.5)

elapsed()

0.5002197999856435

With Timer, you can measure the execution time of any piece of code. In this example, timer holds an instance of the lambda function that computes the time delta, so you need to call timer() to get the final result.

2. **Function-Based Context Managers**:

Python’s generator functions and the contextlib.contextmanager decorator provide an alternative and convenient way to implement the context management protocol. If you decorate an appropriately coded generator function with @contextmanager, then you get a function-based context manager that automatically provides both required methods, `.__enter__()` and `.__exit__()`. This can make your life more pleasant by saving you some boilerplate code.

The general pattern to create a context manager using @contextmanager along with a generator function goes like this:

In [49]:
from contextlib import contextmanager

@contextmanager
def hello_context_manager():
    print("Entering the context...")
    yield "Hello, World!"
    print("Leaving the context...")


with hello_context_manager() as hello:
    print(hello)

Entering the context...
Hello, World!
Leaving the context...


In this example, you can identify two visible sections in hello_context_manager(). Before the yield statement, you have the setup section. There, you can place the code that acquires the managed resources. Everything before the yield runs when the flow of execution enters the context.

After the yield statement, you have the teardown section, in which you can release the resources and do the cleanup. The code after yield runs at the end of the with block. The yield statement itself provides the object that will be assigned to the with target variable.

This implementation and the one that uses the context management protocol are practically equivalent. Depending on which one you find more readable, you might prefer one over the other. 

The @contextmanager decorator reduces the boilerplate required to create a context manager. Instead of writing a whole class with .__enter__() and .__exit__() methods, you just need to implement a generator function with a single yield that produces whatever you want .__enter__() to return.

**Measuring Execution Time, Function-Based Context Manager**

In [52]:
from contextlib import contextmanager
from time import perf_counter

@contextmanager
def timer():
    start = perf_counter()
    yield lambda: perf_counter() - start


In [53]:
from time import sleep

with timer() as elapsed:
    # Time-consuming code goes here...
    sleep(0.5)

elapsed()

0.5002292999997735

The timer function directly yields a lambda function that calculates the elapsed time when called. This lambda function captures the start variable and calls perf_counter() again each time it's invoked to get the current time and calculate the elapsed time.